In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.metrics import accuracy_score, classification_report

In [4]:
# Veriyi okuduk
df = pd.read_csv("../data/data.csv", sep=";", encoding="latin1", decimal=",")

# Modelde kullanmayacağımız sütunları çıkardık. 
df = df.drop(columns=["Chief_complain", "Diagnosis in ED", "Saturation","KTAS duration_min"], errors="ignore")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Group                     1267 non-null   int64 
 1   Sex                       1267 non-null   int64 
 2   Age                       1267 non-null   int64 
 3   Patients number per hour  1267 non-null   int64 
 4   Arrival mode              1267 non-null   int64 
 5   Injury                    1267 non-null   int64 
 6   Mental                    1267 non-null   int64 
 7   Pain                      1267 non-null   int64 
 8   NRS_pain                  1267 non-null   object
 9   SBP                       1267 non-null   object
 10  DBP                       1267 non-null   object
 11  HR                        1267 non-null   object
 12  RR                        1267 non-null   object
 13  BT                        1267 non-null   object
 14  KTAS_RN                 

In [5]:
# NRS_pain sütunundaki #BOŞ! olarak belirtilen , acı hissetmeyen hastaların bulunduğu satırlara 0 ataması yaptık .
df["NRS_pain"] = df["NRS_pain"].replace(["#BOÞ!"], 0)

# Sayısal olması gereken sütunları dönüştürdük
for col in ["DBP", "BT","NRS_pain","SBP","HR","RR"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Eksik veri sayısını yazdırıyoruz
print("Eksik veri sayıları sütun bazında:")
print(df.isnull().sum())  

Eksik veri sayıları sütun bazında:
Group                        0
Sex                          0
Age                          0
Patients number per hour     0
Arrival mode                 0
Injury                       0
Mental                       0
Pain                         0
NRS_pain                     0
SBP                         25
DBP                         29
HR                          20
RR                          22
BT                          18
KTAS_RN                      0
Disposition                  0
KTAS_expert                  0
Error_group                  0
Length of stay_min           0
mistriage                    0
dtype: int64


In [6]:
# Eksik verilerin olduğu satırları modelimizi çok fazla etkilemeyeceği için çıkartıyoruz .
df = df.dropna(subset=["SBP", "DBP", "HR", "RR", "BT"])

In [7]:
def ktas_to_class(ktas_value):
    if ktas_value in [1, 2]:
        return "Kırmızı"
    elif ktas_value == 3:
        return "Sarı"
    elif ktas_value in [4, 5]:
        return "Yeşil"
    else:
        return "Bilinmiyor"

df["ktas_class"] = df["KTAS_RN"].apply(ktas_to_class)

In [8]:
# Risk skoru hesaplıyoruz
def calculate_risk(row):
    risk = 0
    if row["BT"] >= 38: risk += 1
    if row["HR"] >= 100: risk += 1
    if row["Age"] >= 70: risk += 1
    return risk

df["risk_score"] = df.apply(calculate_risk, axis=1)

In [9]:
# Kategorik sütunları one-hot encode ettik
df_encoded = pd.get_dummies(df, columns=["Sex", "Arrival mode", "Injury", "Mental", "Pain"])

# Hedef değişkeni encode ettik
label_encoder = LabelEncoder()
df_encoded["target"] = label_encoder.fit_transform(df["ktas_class"])

df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1233 entries, 0 to 1266
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Group                     1233 non-null   int64  
 1   Age                       1233 non-null   int64  
 2   Patients number per hour  1233 non-null   int64  
 3   NRS_pain                  1233 non-null   int64  
 4   SBP                       1233 non-null   float64
 5   DBP                       1233 non-null   float64
 6   HR                        1233 non-null   float64
 7   RR                        1233 non-null   float64
 8   BT                        1233 non-null   float64
 9   KTAS_RN                   1233 non-null   int64  
 10  Disposition               1233 non-null   int64  
 11  KTAS_expert               1233 non-null   int64  
 12  Error_group               1233 non-null   int64  
 13  Length of stay_min        1233 non-null   int64  
 14  mistriage    

In [10]:

# Model girdilerini ve hedef sütunu ayırıyoruz
drop_cols = ["ktas_class","target","KTAS_RN","KTAS_expert"]
X = df_encoded.drop(columns=drop_cols)
y = df_encoded["target"]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Veriyi eğitim ve test olarak ayırdık
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Modeli eğitiyoruz
model = LogisticRegression(max_iter=1000,solver="lbfgs")
model.fit(X_train, y_train)

##
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report

print("📊 Train Accuracy:", accuracy_score(y_train, train_pred))
print("📊 Test Accuracy:", accuracy_score(y_test, test_pred))

y_pred = model.predict(X_test)

📊 Train Accuracy: 0.6592292089249493
📊 Test Accuracy: 0.6072874493927125


In [11]:
acc = accuracy_score(y_test, y_pred)
print(f"📊 Doğruluk (Accuracy): {acc:.2f}")
print("\n📋 Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

joblib.dump(model, "model.joblib")
joblib.dump(label_encoder, "label_encoder.joblib")
joblib.dump(X.columns.tolist(), "feature_columns.joblib")
print("✅ Model başarıyla kaydedildi.")

📊 Doğruluk (Accuracy): 0.61

📋 Sınıflandırma Raporu:
              precision    recall  f1-score   support

     Kırmızı       0.57      0.27      0.36        45
        Sarı       0.55      0.65      0.59        83
       Yeşil       0.66      0.71      0.68       119

    accuracy                           0.61       247
   macro avg       0.59      0.54      0.55       247
weighted avg       0.61      0.61      0.59       247

✅ Model başarıyla kaydedildi.
